In [1]:
%run "..\general_functions\Extracting Data Functions.ipynb"
%run "..\general_functions\generalFunctions.ipynb"

## Data Parameters

In [2]:
f_name = os.path.join(os.path.dirname(os.getcwd()),"Edgewell Germany Dataset.xlsx")


client_manuf = ["Handel"]
client_brands = ["Handel Eigenmarke"]

decimals = 2
sign = "After"
currency = '€'
currency = ' '+ currency if sign.lower() == 'after' else  currency + ' '

prodORitem = "Product"
categories = ["Total Shaving"]
sectors = ["Female Shaving", "Male Shaving"]
segments = ["Disposables", "System"]
subsegments= ["Disposables", "System Blades", "System Razor"]
subcategories= ["Disposables", "System Blades", "Banded Pack", "Handle"]
 
national = True
customareas= ""
areas = ['CHANNEL','NATIONAL']

regions_RET  = []
channels_RET = []
market_RET = []

regions_CHAN = ["Drug Stores", "Discounter Gesamt",	"Hm/Sm"]
channels_CHAN = []
market_CHAN = []
 
regions_CUST = []
channels_CUST = []
market_CUST = []
data_source = "DATA SOURCE: Trade Panel/Retailer Data | Jan 2025"
 
years = ['2022','2023', "2024"]
end_date = "2025-02-01"
# period = 'P3M'
percent = 100000
percentstr="'00 000"
ValueCutOff = 1000
ManufOn=False
BrandOrTopB="Top Brands"
ManufOrTopC="Top Companies"
sectorInScope = sectors
segmentInScope = segments
subsegmentsInScope=subsegments
subcategoriesInScope=subcategories
# brackets = ['[Base Price Bracket].[Base Price\xa0Bracket]','[Products].[Size Bracket]','[Products].[Portion Count Bracket]']
brackets = ['[Base Price Bracket].[Base Price\xa0Bracket]','[Products].[Size Bracket]']

ISDcolumn='Total Size'# default should be 'Total Size'
direct_parent = {"Sector":"Category",
                "Segment":"Sector",
                "SubSegment":"Segment", 
                "SubCategory":"Segment"}

stringtodic = {"Category":categories,
                "Sector":sectors,
                "Segment":segments, 
                "SubSegment":subsegments}




In [3]:
f_path = Path.cwd()
excel = client.gencache.EnsureDispatch('Excel.Application')
excel.Visible = True  # False
wb = excel.Workbooks.Open(f_name)
ws=wb.Sheets([s.Name for s in wb.Sheets][0])
s_name = [s.Name for s in wb.Sheets][0]
pvtTable = ws.PivotTables(1)

#change report layout
pvtTable.RowAxisLayout(1)   #RowAxisLayout(1) for tabular form
#change pivot table style
#Select from Design tab, try out Medium9 or Medium3
pvtTable.TableStyle2 = "pivotStyleMedium21"
pvtTable.ClearTable()

pvtTable.TableRange2.Cut(ws.Range("A16"))
fieldsNamePosition={}
for i in range(1,pvtTable.CubeFields.Count+1):
    fieldsNamePosition[str(pvtTable.CubeFields(i))]=i

In [4]:
row_list = []
column_list=[]
filter_list=['[Products].[Category]','[Time Logic].[Time Period]','[Scope].[Scope]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Products].[Sector]','[Products].[Segment]','[Products].[SubCategory]','[Products].[SubSegment]']
value_list=['[Measures].[Value Share]','[Measures].[Brand WoB %]','[Measures].[WoB %]','[Measures].[Value Sales IYA]', '[Measures].[Relative Price]']

pvtTable.ClearTable()
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

#Select the filter values for each filter
# pvtTable.PivotFields("[Products].[Category].[Category]").ClearAllFilters()
pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

# pvtTable.PivotFields("[Time Logic].[Time Period].[Time Period]").ClearAllFilters()
pvtTable.PivotFields('[Time Logic].[Time Period].[Time Period]').CurrentPageName = '[Time Logic].[Time Period].&[P12M]'


In [5]:
start_time = time.time()

##### TopLine By Brand

In [6]:
row_list = []
column_list=[]
filter_list=['[Products].[Category]','[Time Logic].[Time Period]','[Scope].[Scope]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Products].[Sector]','[Products].[Segment]','[Products].[SubCategory]','[Products].[SubSegment]']
value_list=['[Measures].[Value Share]','[Measures].[Brand WoB %]','[Measures].[WoB %]','[Measures].[Value Sales IYA]', '[Measures].[Relative Price]']

pvtTable.ClearTable()
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

#Select the filter values for each filter
# pvtTable.PivotFields("[Products].[Category].[Category]").ClearAllFilters()
pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

# pvtTable.PivotFields("[Time Logic].[Time Period].[Time Period]").ClearAllFilters()
pvtTable.PivotFields('[Time Logic].[Time Period].[Time Period]').CurrentPageName = '[Time Logic].[Time Period].&[P12M]'


In [7]:
outputs = {}
for i in brackets:
    print(i)
    row_list=[f'{i}', '[Products].[' +BrandOrTopB+ ']']
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)
    dic_name = f"share_topline_{i.split('.[')[1]}".replace("]", "").replace(" ","_").replace("\xa0","_").lower()
    outputs[dic_name]= {}

    # Slides by Category for Areas = NATIONAL, RETAILER, CHANNEL
    if len(categories) !=0:
        pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
        calculate_category_data(outputs[dic_name])
    calculate_sector_segment_data(outputs[dic_name])
        
    pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].[' +BrandOrTopB+ ']').values())[0]).Orientation = 0
    pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, f'{i}').values())[0]).Orientation = 0


    with open(f"PPA Datasets/{dic_name}.pickle", 'wb') as handle:
        pickle.dump(outputs[dic_name], handle, protocol=pickle.HIGHEST_PROTOCOL)
    


[Base Price Bracket].[Base Price Bracket]
[Products].[Size Bracket]


##### CatScope By Brand

In [8]:

row_list = []
column_list=[]
filter_list=['[Products].[Category]','[Time Logic].[Time Period]','[Scope].[Scope]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]']
value_list=['[Measures].[Value Share]','[Measures].[Brand WoB %]','[Measures].[WoB %]']

pvtTable.ClearTable()
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

#Select the filter values for each filter
# pvtTable.PivotFields("[Products].[Category].[Category]").ClearAllFilters()
pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

# pvtTable.PivotFields("[Time Logic].[Time Period].[Time Period]").ClearAllFilters()
pvtTable.PivotFields('[Time Logic].[Time Period].[Time Period]').CurrentPageName = '[Time Logic].[Time Period].&[P12M]'

pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'

In [9]:
outputs_cat = {}
for i in brackets:
    print(i)
    dic_name_ori = f"share_topline_{i.split('.[')[1]}_catscope".replace("]", "").replace(" ","_").replace("\xa0","_").lower()
    name_new = ''     
    if len(sectors) != 0:
        dic_name= dic_name_ori +"_sectors"
        outputs_cat[dic_name]= {}
        data_frame=outputs_cat[dic_name]
        row_list=[f'{i}','[Products].[Sector]', '[Products].[' +BrandOrTopB+ ']']
        pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)
        calculate_data(data_frame,'')
        pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].[Sector]').values())[0]).Orientation = 0  
        pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].[' +BrandOrTopB+ ']').values())[0]).Orientation = 0
        pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, f'{i}').values())[0]).Orientation = 0
        with open(f"PPA Datasets/{dic_name}.pickle", 'wb') as handle:
            pickle.dump(outputs_cat[dic_name], handle, protocol=pickle.HIGHEST_PROTOCOL)

    if len(segments) != 0:
        dic_name= dic_name_ori +"_segments"
        outputs_cat[dic_name]= {}
        data_frame=outputs_cat[dic_name]
        row_list=[f'{i}','[Products].[Segment]', '[Products].[' +BrandOrTopB+ ']']
        pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)
        calculate_data(data_frame,'')
        pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].[Segment]').values())[0]).Orientation = 0  
        pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].[' +BrandOrTopB+ ']').values())[0]).Orientation = 0
        pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, f'{i}').values())[0]).Orientation = 0
        with open(f"PPA Datasets/{dic_name}.pickle", 'wb') as handle:
            pickle.dump(outputs_cat[dic_name], handle, protocol=pickle.HIGHEST_PROTOCOL)
    if len(subsegments) != 0:
        dic_name= dic_name_ori +"_subsegments"
        outputs_cat[dic_name]= {}
        data_frame=outputs_cat[dic_name]
        row_list=[f'{i}','[Products].[SubSegment]', '[Products].[' +BrandOrTopB+ ']']
        pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)
        calculate_data(data_frame,'')
        pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].[SubSegment]').values())[0]).Orientation = 0 
        pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].[' +BrandOrTopB+ ']').values())[0]).Orientation = 0
        pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, f'{i}').values())[0]).Orientation = 0
        with open(f"PPA Datasets/{dic_name}.pickle", 'wb') as handle:
            pickle.dump(outputs_cat[dic_name], handle, protocol=pickle.HIGHEST_PROTOCOL)
    if len(subcategories) != 0:
        dic_name= dic_name_ori +"_subcategories"
        outputs_cat[dic_name]= {}
        data_frame=outputs_cat[dic_name]
        row_list=[f'{i}','[Products].[SubCategory]', '[Products].[' +BrandOrTopB+ ']']
        pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)
        calculate_data(data_frame,'')
        pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].[SubCategory]').values())[0]).Orientation = 0  
        pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].[' +BrandOrTopB+ ']').values())[0]).Orientation = 0
        pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, f'{i}').values())[0]).Orientation = 0
        with open(f"PPA Datasets/{dic_name}.pickle", 'wb') as handle:
            pickle.dump(outputs_cat[dic_name], handle, protocol=pickle.HIGHEST_PROTOCOL)
                      


[Base Price Bracket].[Base Price Bracket]
[Products].[Size Bracket]


##### CatScope By Manuf

In [10]:
if ManufOn:
    row_list = []
    column_list=[]
    filter_list=['[Products].[Category]','[Time Logic].[Time Period]','[Scope].[Scope]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]']
    value_list=['[Measures].[Value Share]','[Measures].[Company WoB %]','[Measures].[WoB %]']

    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    #Select the filter values for each filter
    # pvtTable.PivotFields("[Products].[Category].[Category]").ClearAllFilters()
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

    # pvtTable.PivotFields("[Time Logic].[Time Period].[Time Period]").ClearAllFilters()
    pvtTable.PivotFields('[Time Logic].[Time Period].[Time Period]').CurrentPageName = '[Time Logic].[Time Period].&[P12M]'

    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'

In [11]:
if ManufOn:
    outputs_cat_manuf = {}
    for i in brackets:
        print(i)
        dic_name_ori = f"share_topline_{i.split('.[')[1]}_catscope_manuf".replace("]", "").replace(" ","_").replace("\xa0","_").lower()
        name_new = ''     
        if len(sectors) != 0:
            dic_name= dic_name_ori +"_sectors"
            outputs_cat_manuf[dic_name]= {}
            data_frame=outputs_cat_manuf[dic_name]
            row_list=[f'{i}','[Products].[Sector]', '[Products].[' +ManufOrTopC+ ']']
            pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)
            calculate_data(data_frame,'')
            pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].[Sector]').values())[0]).Orientation = 0  
            pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].[' +ManufOrTopC+ ']').values())[0]).Orientation = 0
            pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, f'{i}').values())[0]).Orientation = 0
            with open(f"PPA Datasets/{dic_name}.pickle", 'wb') as handle:
                pickle.dump(outputs_cat_manuf[dic_name], handle, protocol=pickle.HIGHEST_PROTOCOL)

        if len(segments) != 0:
            dic_name= dic_name_ori +"_segments"
            outputs_cat_manuf[dic_name]= {}
            data_frame=outputs_cat_manuf[dic_name]
            row_list=[f'{i}','[Products].[Segment]', '[Products].[' +ManufOrTopC+ ']']
            pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)
            calculate_data(data_frame,'')
            pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].[Segment]').values())[0]).Orientation = 0  
            pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].[' +ManufOrTopC+ ']').values())[0]).Orientation = 0
            pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, f'{i}').values())[0]).Orientation = 0
            with open(f"PPA Datasets/{dic_name}.pickle", 'wb') as handle:
                pickle.dump(outputs_cat_manuf[dic_name], handle, protocol=pickle.HIGHEST_PROTOCOL)
        if len(subsegments) != 0:
            dic_name= dic_name_ori +"_subsegments"
            outputs_cat_manuf[dic_name]= {}
            data_frame=outputs_cat_manuf[dic_name]
            row_list=[f'{i}','[Products].[SubSegment]', '[Products].[' +ManufOrTopC+ ']']
            pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)
            calculate_data(data_frame,'')
            pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].[SubSegment]').values())[0]).Orientation = 0 
            pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].[' +ManufOrTopC+ ']').values())[0]).Orientation = 0
            pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, f'{i}').values())[0]).Orientation = 0
            with open(f"PPA Datasets/{dic_name}.pickle", 'wb') as handle:
                pickle.dump(outputs_cat_manuf[dic_name], handle, protocol=pickle.HIGHEST_PROTOCOL)
        if len(subcategories) != 0:
            dic_name= dic_name_ori +"_subcategories"
            outputs_cat_manuf[dic_name]= {}
            data_frame=outputs_cat_manuf[dic_name]
            row_list=[f'{i}','[Products].[SubCategory]', '[Products].[' +ManufOrTopC+ ']']
            pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)
            calculate_data(data_frame,'')
            pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].[SubCategory]').values())[0]).Orientation = 0  
            pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].[' +ManufOrTopC+ ']').values())[0]).Orientation = 0
            pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, f'{i}').values())[0]).Orientation = 0
            with open(f"PPA Datasets/{dic_name}.pickle", 'wb') as handle:
                pickle.dump(outputs_cat_manuf[dic_name], handle, protocol=pickle.HIGHEST_PROTOCOL)
                        

In [12]:
if ManufOn == False:
    outputs_cat_manuf = {}
    for i in brackets:
        dic_name_ori = f"share_topline_{i.split('.[')[1]}_catscope_manuf".replace("]", "").replace(" ","_").replace("\xa0","_").lower()
        name_new = ''     
        if len(sectors) != 0:
            dic_name= dic_name_ori +"_sectors"
            outputs_cat_manuf[dic_name]= {}
            data_frame=outputs_cat_manuf[dic_name]
            with open(f"PPA Datasets/{dic_name}.pickle", 'wb') as handle:
                pickle.dump(outputs_cat_manuf[dic_name], handle, protocol=pickle.HIGHEST_PROTOCOL)
        if len(segments) != 0:
            dic_name= dic_name_ori +"_segments"
            outputs_cat_manuf[dic_name]= {}
            data_frame=outputs_cat_manuf[dic_name]
            with open(f"PPA Datasets/{dic_name}.pickle", 'wb') as handle:
                pickle.dump(outputs_cat_manuf[dic_name], handle, protocol=pickle.HIGHEST_PROTOCOL)
        if len(subsegments) != 0:
            dic_name= dic_name_ori +"_subsegments"
            outputs_cat_manuf[dic_name]= {}
            data_frame=outputs_cat_manuf[dic_name]
            with open(f"PPA Datasets/{dic_name}.pickle", 'wb') as handle:
                pickle.dump(outputs_cat_manuf[dic_name], handle, protocol=pickle.HIGHEST_PROTOCOL)
        if len(subcategories) != 0:
            dic_name= dic_name_ori +"_subcategories"
            outputs_cat_manuf[dic_name]= {}
            data_frame=outputs_cat_manuf[dic_name]
            with open(f"PPA Datasets/{dic_name}.pickle", 'wb') as handle:
                pickle.dump(outputs_cat_manuf[dic_name], handle, protocol=pickle.HIGHEST_PROTOCOL)


##### Parent Scope By Brand

In [13]:
row_list = []
column_list=[]
filter_list=['[Products].[Category]','[Time Logic].[Time Period]','[Scope].[Scope]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]']
value_list=['[Measures].[Value Share]','[Measures].[Brand WoB %]','[Measures].[WoB %]','[Measures].[Value Sales]']

pvtTable.ClearTable()
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

#Select the filter values for each filter
# pvtTable.PivotFields("[Products].[Category].[Category]").ClearAllFilters()
pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

# pvtTable.PivotFields("[Time Logic].[Time Period].[Time Period]").ClearAllFilters()
pvtTable.PivotFields('[Time Logic].[Time Period].[Time Period]').CurrentPageName = '[Time Logic].[Time Period].&[P12M]'

pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'

In [14]:
outputs_parent = {}
for i in brackets:
    print(i)
    dic_name_ori = f"share_topline_{i.split('.[')[1]}_parentscope".replace("]", "").replace(" ","_").replace("\xa0","_").lower()
    name_new = ''     
    if len(segments) != 0:
        dic_name= dic_name_ori +"_segments"
        outputs_parent[dic_name]= {}
        data_frame=outputs_parent[dic_name]
        row_list=[f'{i}','[Products].['+direct_parent["Segment"]+']','[Products].[Segment]', '[Products].[' +BrandOrTopB+ ']']
        pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&['+direct_parent["Segment"]+']'
        calculate_data(data_frame,'')
        pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].['+direct_parent["Segment"]+']').values())[0]).Orientation = 0  
        pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].[Segment]').values())[0]).Orientation = 0  
        pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].[' +BrandOrTopB+ ']').values())[0]).Orientation = 0
        pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, f'{i}').values())[0]).Orientation = 0
        with open(f"PPA Datasets/{dic_name}.pickle", 'wb') as handle:
            pickle.dump(outputs_parent[dic_name], handle, protocol=pickle.HIGHEST_PROTOCOL)
    if len(subsegments) != 0:
        dic_name= dic_name_ori +"_subsegments"
        outputs_parent[dic_name]= {}
        data_frame=outputs_parent[dic_name]
        row_list=[f'{i}','[Products].['+direct_parent["SubSegment"]+']','[Products].[SubSegment]', '[Products].[' +BrandOrTopB+ ']']
        pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&['+direct_parent["SubSegment"]+']'
        calculate_data(data_frame,'')
        pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].['+direct_parent["SubSegment"]+']').values())[0]).Orientation = 0 
        pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].[SubSegment]').values())[0]).Orientation = 0 
        pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].[' +BrandOrTopB+ ']').values())[0]).Orientation = 0
        pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, f'{i}').values())[0]).Orientation = 0
        with open(f"PPA Datasets/{dic_name}.pickle", 'wb') as handle:
            pickle.dump(outputs_parent[dic_name], handle, protocol=pickle.HIGHEST_PROTOCOL)
    if len(subcategories) != 0:
        dic_name= dic_name_ori +"_subcategories"
        outputs_parent[dic_name]= {}
        data_frame=outputs_parent[dic_name]
        row_list=[f'{i}','[Products].['+direct_parent["SubCategory"]+']','[Products].[SubCategory]', '[Products].[' +BrandOrTopB+ ']']
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&['+direct_parent["SubCategory"]+']'
        pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)
        calculate_data(data_frame,'')
        pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].['+direct_parent["SubCategory"]+']').values())[0]).Orientation = 0  
        pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].[SubCategory]').values())[0]).Orientation = 0  
        pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].[' +BrandOrTopB+ ']').values())[0]).Orientation = 0
        pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, f'{i}').values())[0]).Orientation = 0
        with open(f"PPA Datasets/{dic_name}.pickle", 'wb') as handle:
            pickle.dump(outputs_parent[dic_name], handle, protocol=pickle.HIGHEST_PROTOCOL)

[Base Price Bracket].[Base Price Bracket]
[Products].[Size Bracket]


##### Parent Scope By Manuf

In [15]:
if ManufOn:
    row_list = []
    column_list=[]
    filter_list=['[Products].[Category]','[Time Logic].[Time Period]','[Scope].[Scope]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]']
    value_list=['[Measures].[Value Share]','[Measures].[Company WoB %]','[Measures].[WoB %]','[Measures].[Value Sales]']

    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    #Select the filter values for each filter
    # pvtTable.PivotFields("[Products].[Category].[Category]").ClearAllFilters()
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

    # pvtTable.PivotFields("[Time Logic].[Time Period].[Time Period]").ClearAllFilters()
    pvtTable.PivotFields('[Time Logic].[Time Period].[Time Period]').CurrentPageName = '[Time Logic].[Time Period].&[P12M]'

    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'

In [16]:
if ManufOn:
    outputs_parent_manuf = {}
    for i in brackets:
        print(i)
        dic_name_ori = f"share_topline_{i.split('.[')[1]}_parentscope_manuf".replace("]", "").replace(" ","_").replace("\xa0","_").lower()
        name_new = ''     
        if len(segments) != 0:
            dic_name= dic_name_ori +"_segments"
            outputs_parent_manuf[dic_name]= {}
            data_frame=outputs_parent_manuf[dic_name]
            row_list=[f'{i}','[Products].['+direct_parent["Segment"]+']','[Products].[Segment]', '[Products].[' +ManufOrTopC+ ']']
            pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)
            pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&['+direct_parent["Segment"]+']'
            calculate_data(data_frame,'')
            pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].['+direct_parent["Segment"]+']').values())[0]).Orientation = 0  
            pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].[Segment]').values())[0]).Orientation = 0  
            pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].[' +ManufOrTopC+ ']').values())[0]).Orientation = 0
            pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, f'{i}').values())[0]).Orientation = 0
            with open(f"PPA Datasets/{dic_name}.pickle", 'wb') as handle:
                pickle.dump(outputs_parent_manuf[dic_name], handle, protocol=pickle.HIGHEST_PROTOCOL)
        if len(subsegments) != 0:
            dic_name= dic_name_ori +"_subsegments"
            outputs_parent_manuf[dic_name]= {}
            data_frame=outputs_parent_manuf[dic_name]
            row_list=[f'{i}','[Products].['+direct_parent["SubSegment"]+']','[Products].[SubSegment]', '[Products].[' +ManufOrTopC+ ']']
            pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)
            pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&['+direct_parent["SubSegment"]+']'
            calculate_data(data_frame,'')
            pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].['+direct_parent["SubSegment"]+']').values())[0]).Orientation = 0 
            pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].[SubSegment]').values())[0]).Orientation = 0 
            pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].[' +ManufOrTopC+ ']').values())[0]).Orientation = 0
            pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, f'{i}').values())[0]).Orientation = 0
            with open(f"PPA Datasets/{dic_name}.pickle", 'wb') as handle:
                pickle.dump(outputs_parent_manuf[dic_name], handle, protocol=pickle.HIGHEST_PROTOCOL)
        if len(subcategories) != 0:
            dic_name= dic_name_ori +"_subcategories"
            outputs_parent_manuf[dic_name]= {}
            data_frame=outputs_parent_manuf[dic_name]
            row_list=[f'{i}','[Products].['+direct_parent["SubCategory"]+']','[Products].[SubCategory]', '[Products].[' +ManufOrTopC+ ']']
            pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&['+direct_parent["SubCategory"]+']'
            pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)
            calculate_data(data_frame,'')
            pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].['+direct_parent["SubCategory"]+']').values())[0]).Orientation = 0  
            pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].[SubCategory]').values())[0]).Orientation = 0  
            pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].[' +ManufOrTopC+ ']').values())[0]).Orientation = 0
            pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, f'{i}').values())[0]).Orientation = 0
            with open(f"PPA Datasets/{dic_name}.pickle", 'wb') as handle:
                pickle.dump(outputs_parent_manuf[dic_name], handle, protocol=pickle.HIGHEST_PROTOCOL)
                        


In [17]:
if ManufOn == False:
    outputs_parent_manuf = {}
    for i in brackets:
        print(i)
        dic_name_ori = f"share_topline_{i.split('.[')[1]}_parentscope_manuf".replace("]", "").replace(" ","_").replace("\xa0","_").lower()
        name_new = ''     
        if len(segments) != 0:
            dic_name= dic_name_ori +"_segments"
            outputs_parent_manuf[dic_name]= {}
            data_frame=outputs_parent_manuf[dic_name]
            with open(f"PPA Datasets/{dic_name}.pickle", 'wb') as handle:
                pickle.dump(outputs_parent_manuf[dic_name], handle, protocol=pickle.HIGHEST_PROTOCOL)
                        
        if len(subsegments) != 0:
            dic_name= dic_name_ori +"_subsegments"
            outputs_parent_manuf[dic_name]= {}
            data_frame=outputs_parent_manuf[dic_name]
            with open(f"PPA Datasets/{dic_name}.pickle", 'wb') as handle:
                pickle.dump(outputs_parent_manuf[dic_name], handle, protocol=pickle.HIGHEST_PROTOCOL)
                        
        if len(subcategories) != 0:
            dic_name= dic_name_ori +"_subcategories"
            outputs_parent_manuf[dic_name]= {}
            data_frame=outputs_parent_manuf[dic_name]
            with open(f"PPA Datasets/{dic_name}.pickle", 'wb') as handle:
                pickle.dump(outputs_parent_manuf[dic_name], handle, protocol=pickle.HIGHEST_PROTOCOL)
                        
    

[Base Price Bracket].[Base Price Bracket]
[Products].[Size Bracket]


## Inter-size Discount

In [18]:
if ISDcolumn != 'Total Size':
    row_list=['[Products].[Variant]','[Products].[Total Size]','[Products].['+ISDcolumn+']']
else:
    row_list=['[Products].[Variant]','[Products].[Total Size]']
column_list=[]
filter_list=['[Products].[Category]','[Time Logic].[Time Period]','[Scope].[Scope]','[Products].[Top Brands]','[Market].[Area]','[Market].[Region]','[Market].[Channel]',"[Market].[Market]"]

value_list=['[Measures].[Base Price/Unit]', '[Measures].[Value Sales]','[Measures].[Unit Sales]','[Measures].[Value Sales IYA]', '[Measures].[VSOD]']

pvtTable.ClearTable()
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

# Select the filter values for each filter
# pvtTable.PivotFields("[Products].[Category].[Category]").ClearAllFilters()
pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

# pvtTable.PivotFields("[Time Logic].[Time Period].[Time Period]").ClearAllFilters()
pvtTable.PivotFields('[Time Logic].[Time Period].[Time Period]').CurrentPageName = '[Time Logic].[Time Period].&[P3M]'

In [19]:
inter_size = {}
if len(categories)!=0:
    # pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'

    for brand in client_brands:
        pvtTable.PivotFields('[Products].[Top Brands].[Top Brands]').ClearAllFilters()
        pvtTable.PivotFields('[Products].[Top Brands].[Top Brands]').CurrentPageName = f'[Products].[Top Brands].&[{brand}]'
        calculate_category_data(inter_size,' | '+f'{brand}')
with open('PPA Datasets/inter_size.pickle', 'wb') as handle:
    pickle.dump(inter_size, handle, protocol=pickle.HIGHEST_PROTOCOL)

##### TopLine By Manufacture

In [20]:
if ManufOn:
    row_list = []
    column_list=[]
    filter_list=['[Products].[Category]','[Time Logic].[Time Period]','[Scope].[Scope]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Products].[Sector]','[Products].[Segment]','[Products].[SubCategory]','[Products].[SubSegment]']
    value_list=['[Measures].[Value Share]','[Measures].[Company WoB %]','[Measures].[Value Sales IYA]', '[Measures].[Relative Price]']

    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    #Select the filter values for each filter
    # pvtTable.PivotFields("[Products].[Category].[Category]").ClearAllFilters()
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'
    # pvtTable.PivotFields("[Time Logic].[Time Period].[Time Period]").ClearAllFilters()
    pvtTable.PivotFields('[Time Logic].[Time Period].[Time Period]').CurrentPageName = '[Time Logic].[Time Period].&[P12M]'


In [21]:
if ManufOn:
    outputs_manuf = {}
    for i in brackets:
        row_list=[f'{i}', '[Products].[' +ManufOrTopC+ ']']
        print(i)
        pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)
        dic_name = f"share_topline_{i.split('.[')[1]}_manuf".replace("]", "").replace(" ","_").replace("\xa0","_").lower()
        outputs_manuf[dic_name]= {}

        # # Slides by Category for Areas = NATIONAL, RETAILER, CHANNEL
        if len(categories) !=0:
            pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
            pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
            calculate_category_data(outputs_manuf[dic_name])
        calculate_sector_segment_data(outputs_manuf[dic_name])

        pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].[' +ManufOrTopC+ ']').values())[0]).Orientation = 0
        pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, f'{i}').values())[0]).Orientation = 0
    
        with open(f"PPA Datasets/{dic_name}.pickle", 'wb') as handle:
            pickle.dump(outputs_manuf[dic_name], handle, protocol=pickle.HIGHEST_PROTOCOL)
        
            


In [22]:
if ManufOn == False:
    outputs_manuf = {}
    for i in brackets:
        dic_name = f"share_topline_{i.split('.[')[1]}_manuf".replace("]", "").replace(" ","_").replace("\xa0","_").lower()
        outputs_manuf[dic_name]= {}
        with open(f"PPA Datasets/{dic_name}.pickle", 'wb') as handle:
            pickle.dump(outputs_manuf[dic_name], handle, protocol=pickle.HIGHEST_PROTOCOL)
        


In [23]:
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Script started at: {time.ctime(start_time)}")   
print(f"Script ended at: {time.ctime(end_time)}")
print(f"Elapsed time: {elapsed_time:.2f} seconds")

Script started at: Thu Apr 17 15:46:30 2025
Script ended at: Thu Apr 17 15:59:08 2025
Elapsed time: 757.89 seconds


In [24]:
1397.31 

1397.31